In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import pymatgen
import pymatgen.io.xyz
import itertools as it
#targets = ((6,7,7,6,5,5), (7,7,5,5,6,6), (7,7,5,6,5,6), (7,6,7,5,5,6), (5,6,5,7,6,7), (6,6,7,5,5,7), (6,6,5,7,7,5), (7,6,5,7,5,6), (5, 6, 7, 5, 7, 6), (6,6,7,5,7,5), (6,7,5,6,7,5))

In [2]:
def detect_automorphisms(filename):
    xyz = pymatgen.io.xyz.XYZ.from_file(filename)
    psa = pymatgen.symmetry.analyzer.PointGroupAnalyzer(xyz.molecule)
    
    m = xyz.molecule.get_centered_molecule()
    carbons = np.where(np.array(m.atomic_numbers, dtype=np.int) == 6)[0]
    
    operations = psa.get_symmetry_operations()
    mapping = np.zeros((len(carbons), len(operations)), dtype=np.int)
    for opidx, op in enumerate(operations):
        for bidx, base in enumerate(carbons):
            ds = np.linalg.norm(op.operate(m.cart_coords[base]) - m.cart_coords[carbons], axis=1)
            onto = np.argmin(ds)
            if ds[onto] > 1e-3:
                raise ValueError('Irregular geometry')
            mapping[bidx, opidx] = onto
    
    return mapping
def two_at_once(based_on, mappings):
    modifiable = sorted(np.where(np.array(based_on) == 6)[0])
    for i in modifiable:
        for j in modifiable:
            if i == j:
                continue
            zsnew = np.array(based_on).copy()
            zsnew[i] = 5
            zsnew[j] = 7
            
            if is_canonical(zsnew, mappings):
                yield zsnew
def make_canonical(zs, mappings):
    zs = np.array(zs)
    #if len(np.where(zs == 7)[0]) > len(np.where(zs == 5)[0]):
    #    return False
    permutations = zs[mappings].T
    o = np.lexsort(permutations.T)
    canonical = permutations[o[0]]
    return canonical
def is_canonical(zs, mappings):
    return (zs == make_canonical(zs, mappings)).all()

In [3]:
def enumerate_next_bn(based_on):
    mappings = detect_automorphisms("naphtalene.xyz")
    
    res = []
    for k in two_at_once(based_on, mappings):
        res.append(tuple(k))
    return set(res)

In [4]:
bn1 = enumerate_next_bn((6,6,6,6,6,6,6,6,6,6))
bn2 = set.union(*[enumerate_next_bn(_) for _ in bn1])
bn3 = set.union(*[enumerate_next_bn(_) for _ in bn2])
bn4 = set.union(*[enumerate_next_bn(_) for _ in bn3])
bn5 = set.union(*[enumerate_next_bn(_) for _ in bn4])

In [5]:
len(bn1), len(bn2), len(bn3), len(bn4), len(bn5)

(23, 320, 1026, 801, 66)

In [21]:
# ref: 23, 330, 1056, 810, 66

2236